# Scraping Magalu

### Pré-requisitos
- Selenium
- BeautifulSoup
- Pandas
- Unidecode

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from unidecode import unidecode

### Função para pesquisar e personalizar a URL

In [2]:
def personalizar_url(termo_de_pesquisa):
    """
    Função para pesquisar produtos e personalizar a url de pesquisa
    IN: 'smart tv 4k': str
    OUT: https://www.magazineluiza.com.br/busca/iphone 11/?page={}
    """
    template = 'https://www.magazineluiza.com.br/busca/{}/'
    termo_de_pesquisa.replace('', '+')
    
    # Adicionando o query string da pesquisa na URL
    url = template.format(termo_de_pesquisa)
    
    # Adicionando o query string da página na URL
    url += '?page={}'
    
    return url

## Adicionando Opções ao Webdriver

In [3]:
options = webdriver.ChromeOptions() 
options.add_argument('--disable-blink-features=AutomationControlled') #o argumento que fez a diferença foi esse
driver = webdriver.Chrome(options=options, executable_path=r'C:\SeleniumDrivers\chromedriver.exe')

### Scraping Smartphones

In [9]:
url = personalizar_url('smartphone').format(1)
driver = webdriver.Chrome()
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'html.parser')

### Extração da Lista de Produtos

In [10]:
lista_de_produtos = soup.find_all('li', {'class':'sc-eCVOVf loRbcV'})
len(lista_de_produtos)

59

### Prototipando a extração de um registro

In [12]:
item = lista_de_produtos[0]

### Extração do Título do Produto

In [13]:
titulo_produto = item.a.h2.text.strip()
titulo_produto

'iPhone 11 Apple 64GB Branco 6,1” 12MP iOS'

### Extração URL dos Detalhes do Produto

In [14]:
rota_detalhes_produto = item.a.get('href')
rota_detalhes_produto

'/iphone-11-apple-64gb-branco-61-12mp-ios/p/155614100/te/ip11/'

In [15]:
url_detalhes_produto = f'https://www.magazineluiza.com.br{rota_detalhes_produto}'
url_detalhes_produto

'https://www.magazineluiza.com.br/iphone-11-apple-64gb-branco-61-12mp-ios/p/155614100/te/ip11/'

### Extração do Preço

In [16]:
preco_produto = item.find('p', {'data-testid':'price-value'}).text.strip()
preco_produto

'R$\xa03.719,07'

#### Problema com \xa0.
Resolução:
https://stackoverflow.com/questions/26068832/how-to-remove-this-xa0-from-a-string-in-python

In [17]:
preco_produto = item.find('p', {'data-testid':'price-value'}).text.strip()
preco_produto = unidecode(preco_produto)
preco_produto

'R$ 3.719,07'

#### Extração Parte Inteira do Preço

In [18]:
preco_produto_float = float(preco_produto.split()[-1].replace('.','').replace(',','.'))
preco_produto_float

3719.07

### Extraindo Avaliação

Conta-se a avaliação na magalu ao contar o número de estrelas completas -> StarIcon

In [19]:
avaliacao = len(item.find_all('use', {'xlink:href':'#StarIcon'}))
avaliacao

4

### Extraindo Quantidade de Avaliações

In [20]:
quantidade_avaliacoes = int(item.find('span', {'format':'count'}).text.strip())
quantidade_avaliacoes

58

## Encapsulando o Protótipo em Funções
Isto torna nosso código mais limpo e reutilizável

In [21]:
def extrair_titulo_produto(produto):
    return produto.a.h2.text.strip()

def extrair_url_produto(produto):
    rota_detalhes_produto = produto.a.get('href')
    return f'https://www.magazineluiza.com.br{rota_detalhes_produto}'

def extrair_preco_produto(produto):
    preco_produto = produto.find('p', {'data-testid':'price-value'}).text.strip()
    preco_produto = unidecode(preco_produto)
    preco_produto_float = float(preco_produto.split()[-1].replace('.','').replace(',','.'))
    return preco_produto_float
    
def extrair_avaliacao(produto):
    return len(produto.find_all('use', {'xlink:href':'#StarIcon'})) 

def extrair_quantidade_avaliacoes(produto):
    return int(produto.find('span', {'format':'count'}).text.strip())

### Agregando os códigos para a função de extrair informações do produto

In [22]:
def extrair_informacoes_produto(produto):
    titulo = extrair_titulo_produto(produto)
    preco = extrair_preco_produto(produto)
    avaliacao = extrair_avaliacao(produto)
    quantidade_avaliacoes = extrair_quantidade_avaliacoes(produto)
    url = extrair_url_produto(produto)
    
    return (titulo, preco, avaliacao, quantidade_avaliacoes, url)

#### Testando a função

In [23]:
print(extrair_informacoes_produto(item))

('iPhone 11 Apple 64GB Branco 6,1” 12MP iOS', 3719.07, 4, 58, 'https://www.magazineluiza.com.br/iphone-11-apple-64gb-branco-61-12mp-ios/p/155614100/te/ip11/')


Deu tudo certo até agora 😁

### Testando para saber se pega todos os itens da página com sucesso

In [24]:
lista_produtos = []
resultado_da_pesquisa = soup.find_all('li', {'class':'sc-eCVOVf loRbcV'})

for produto in resultado_da_pesquisa:
    lista_produtos.append(extrair_informacoes_produto(produto))

AttributeError: 'NoneType' object has no attribute 'text'

Deu errado ao extrair a quantidade de avaliações 😅

Podemos corrigir isso através do tratamento de exceções

In [26]:
def extrair_titulo_produto(produto):
    return produto.a.h2.text.strip()

def extrair_url_produto(produto):
    rota_detalhes_produto = produto.a.get('href')
    return f'https://www.magazineluiza.com.br{rota_detalhes_produto}'

def extrair_preco_produto(produto):
    try:
        preco_produto = produto.find('p', {'data-testid':'price-value'}).text.strip()
        preco_produto = unidecode(preco_produto)
        preco_produto_float = float(preco_produto.split()[-1].replace('.','').replace(',','.'))
        return preco_produto_float
    except AttributeError:
        return
    
def extrair_avaliacao(produto):
    return len(produto.find_all('use', {'xlink:href':'#StarIcon'})) 

def extrair_quantidade_avaliacoes(produto):
    try:
        return int(produto.find('span', {'format':'count'}).text.strip())
    except AttributeError:
        return 0

Testando novamente...

In [27]:
lista_produtos = []
resultado_da_pesquisa = soup.find_all('li', {'class':'sc-eCVOVf loRbcV'})
for produto in resultado_da_pesquisa:
    titulo_produto = produto.a.h2.text.strip()
    print(titulo_produto)
    lista_produtos.append(extrair_informacoes_produto(produto))

iPhone 11 Apple 64GB Branco 6,1” 12MP iOS
iPhone 11 Apple 64GB Roxo 6,1” 12MP iOS
Smartphone Samsung Galaxy S20 FE 128GB Cloud Mint
iPhone 11 Apple 64GB Preto 6,1” 12MP iOS
Smartphone Samsung Galaxy S20 FE 128GB Cloud White
Smartphone Motorola Moto G30 128GB White Lilac 4G
Smartphone Samsung Galaxy A12 64GB Branco 4G
Smartphone Samsung Galaxy S20 FE 128GB Cloud Navy
Smartphone Samsung Galaxy M12 64GB Verde 4G
Smartphone Samsung Galaxy A02s 32GB Vermelho 4G
Smartphone Samsung Galaxy M62 128GB Azul
Smartphone Motorola Moto E7 Power 32GB Vermelho
Smartphone Samsung Galaxy A12 64GB Azul 4G
Smartphone Samsung Galaxy A02s 32GB Azul 4G
Smartphone Samsung Galaxy A12 64GB Vermelho 4G
Smartphone Motorola Moto G9 Play 64GB 4GB RAM Câmera Tripla 48MP Tela 6.5" - Azul Safira
Smartphone Samsung Galaxy M12 64GB Azul 4G
Smartphone Motorola Moto G20 64GB 4G Tela 6.5" Câmera Quádupla 48MP 8MP 2MP 2MP Frontal 13MP Azul
iPhone 11 Apple 128GB Amarelo 6,1” 12MP iOS
iPhone 11 Apple 128GB Preto 6,1” 12MP iOS


In [28]:
print(lista_produtos[:5])

[('iPhone 11 Apple 64GB Branco 6,1” 12MP iOS', 3719.07, 4, 58, 'https://www.magazineluiza.com.br/iphone-11-apple-64gb-branco-61-12mp-ios/p/155614100/te/ip11/'), ('iPhone 11 Apple 64GB Roxo 6,1” 12MP iOS', 3719.07, 5, 1, 'https://www.magazineluiza.com.br/iphone-11-apple-64gb-roxo-61-12mp-ios/p/155610900/te/ip11/'), ('Smartphone Samsung Galaxy S20 FE 128GB Cloud Mint', 2249.0, 4, 48, 'https://www.magazineluiza.com.br/smartphone-samsung-galaxy-s20-fe-128gb-cloud-mint-4g-6gb-ram-tela-65-cam-tripla-selfie-32mp/p/155630400/te/gs2f/'), ('iPhone 11 Apple 64GB Preto 6,1” 12MP iOS', 3719.07, 4, 80, 'https://www.magazineluiza.com.br/iphone-11-apple-64gb-preto-61-12mp-ios/p/155610500/te/ip11/'), ('Smartphone Samsung Galaxy S20 FE 128GB Cloud White', 2249.0, 4, 51, 'https://www.magazineluiza.com.br/smartphone-samsung-galaxy-s20-fe-128gb-cloud-white-4g-6gb-ram-tela-65-cam-tripla-selfie-32mp/p/155630000/te/gs2f/')]


Agora deu tudo certo! 😎

### Encapsulamento da Função de Extrair Produtos da Página

In [29]:
def extrair_produtos_pagina():
    resultado_da_pesquisa = soup.find_all('li', {'class':'sc-eCVOVf loRbcV'})
    return [extrair_informacoes_produto(produto) for produto in resultado_da_pesquisa if extrair_informacoes_produto(produto)]

Testando a função encapsulada...

In [30]:
extrair_produtos_pagina()[:5]

[('iPhone 11 Apple 64GB Branco 6,1” 12MP iOS',
  3719.07,
  4,
  58,
  'https://www.magazineluiza.com.br/iphone-11-apple-64gb-branco-61-12mp-ios/p/155614100/te/ip11/'),
 ('iPhone 11 Apple 64GB Roxo 6,1” 12MP iOS',
  3719.07,
  5,
  1,
  'https://www.magazineluiza.com.br/iphone-11-apple-64gb-roxo-61-12mp-ios/p/155610900/te/ip11/'),
 ('Smartphone Samsung Galaxy S20 FE 128GB Cloud Mint',
  2249.0,
  4,
  48,
  'https://www.magazineluiza.com.br/smartphone-samsung-galaxy-s20-fe-128gb-cloud-mint-4g-6gb-ram-tela-65-cam-tripla-selfie-32mp/p/155630400/te/gs2f/'),
 ('iPhone 11 Apple 64GB Preto 6,1” 12MP iOS',
  3719.07,
  4,
  80,
  'https://www.magazineluiza.com.br/iphone-11-apple-64gb-preto-61-12mp-ios/p/155610500/te/ip11/'),
 ('Smartphone Samsung Galaxy S20 FE 128GB Cloud White',
  2249.0,
  4,
  51,
  'https://www.magazineluiza.com.br/smartphone-samsung-galaxy-s20-fe-128gb-cloud-white-4g-6gb-ram-tela-65-cam-tripla-selfie-32mp/p/155630000/te/gs2f/')]

Não acredito que tudo até agora rodou liso feito manteiga 🧈

## Função Para Extrair o Número da Última Página

In [31]:
paginas = soup.find_all('a', {'type':'page'})[-1].text
paginas

'17'

In [32]:
def numero_ultima_pagina():
    paginas = soup.find_all('a', {'type':'page'})
    return int(paginas[-1].text)

## Função de Extrair Produtos de Várias Páginas

In [33]:
def extrair_produtos(termo_de_pesquisa=''):
    # Instanciando o driver do navegador
    options = webdriver.ChromeOptions() 
    options.add_argument('--disable-blink-features=AutomationControlled')
    driver = webdriver.Chrome(options=options, executable_path=r'C:\SeleniumDrivers\chromedriver.exe')
    # personalizando a url com o termo de pesquisa
    url = personalizar_url(termo_de_pesquisa)
    
    lista_produtos = []
    for pagina in range(1, numero_ultima_pagina() + 1):
        driver.get(url.format(pagina))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        resultados = soup.find_all('li', {'class':'sc-eCVOVf loRbcV'})
        for produto in resultados:
            informacao = extrair_informacoes_produto(produto)
            if informacao:
                lista_produtos.append(informacao)
    driver.close()
    return lista_produtos

In [34]:
lista_smartphones = extrair_produtos('smartphone')

In [35]:
len(lista_smartphones)

714

In [41]:
lista_smartphones[:3]

[('Smartphone Xiaomi Redmi 8A 32GB Azul Ocean Blue',
  None,
  5,
  0,
  'https://www.magazineluiza.com.br/smartphone-xiaomi-redmi-8a-32gb-azul-ocean-blue-2gb-ram-tela-622-cam-12mp-cam-selfie-8mp/p/225200300/te/rem8/'),
 ('Smartphone Xiaomi Redmi 8A 32GB Vermelho Sunset',
  None,
  5,
  0,
  'https://www.magazineluiza.com.br/smartphone-xiaomi-redmi-8a-32gb-vermelho-sunset-red-2gb-ram-tela-622-cam-12mp-cam-selfie-8mp/p/225200100/te/rem8/'),
 ('Smartphone Samsung Galaxy S20 FE 256GB Cloud Navy',
  2699.1,
  4,
  4,
  'https://www.magazineluiza.com.br/smartphone-samsung-galaxy-s20-fe-256gb-cloud-navy-4g-8gb-ram-tela-65-cam-tripla-selfie-32mp/p/155628600/te/galx/')]

A função está funcionando como esperado

## Salvando dados em csv com pandas

In [37]:
dataset = pd.DataFrame(lista_smartphones, columns = ['descricao', 'preco', 'avaliacao', 'quantidade_avaliacoes', 'URL'])

In [38]:
dataset

,descricao,preco,avaliacao,quantidade_avaliacoes,URL
0,Smartphone Xiaomi Redmi 8A 32GB Azul Ocean Blue,NaN,5,0,https://www.magazineluiza.com.br/smartphone-xi...
1,Smartphone Xiaomi Redmi 8A 32GB Vermelho Sunset,NaN,5,0,https://www.magazineluiza.com.br/smartphone-xi...
2,Smartphone Samsung Galaxy S20 FE 256GB Cloud Navy,2699.10,4,4,https://www.magazineluiza.com.br/smartphone-sa...
3,Smartphone Samsung Galaxy Note 20 256GB Mystic,3149.10,4,4,https://www.magazineluiza.com.br/smartphone-sa...
4,Smartphone Samsung Galaxy A03s 64GB Vermelho 4G,944.10,5,0,https://www.magazineluiza.com.br/smartphone-sa...
...,...,...,...,...,...
709,Smartphone Huawei P40 Dual Sim 128GB - 8GB RAM...,4389.00,5,1,https://www.magazineluiza.com.br/smartphone-hu...
710,Celular Samsung Galaxy A32 Azul 128GB 4GB RAM ...,1549.00,5,0,https://www.magazineluiza.com.br/celular-samsu...
711,Celular Multilaser E Lite 32GB Dual P9127,463.05,5,0,https://www.magazineluiza.com.br/celular-multi...
712,"Smartphone Xiaomi Redmi 9 Tela 6,53"" 4GB 64GB ...",1367.10,5,0,https://www.magazineluiza.com.br/smartphone-xi...


In [39]:
pesquisa = 'smartphone'

In [40]:
dataset.to_csv(f'output/dataset_{pesquisa}.csv'.replace(' ','_'), sep=';', index = False, encoding = 'utf-8-sig')